In [2]:
import pandas as pd

In [3]:
school = pd.read_csv("../data/curated/school_df_21_preprocessed.csv")
park = pd.read_csv("../data/curated/park.csv")

In [4]:
school

,Unnamed: 0,Education_Sector,School_Name,School_Type,Address_Line_1,Address_Town,Address_Postcode,LGA_Name,X,Y
0,0,Government,Alberton Primary School,Primary,21 Thomson Street,Alberton,3971,Wellington (S),146.666601,-38.617713
1,1,Government,Allansford and District Primary School,Primary,Frank Street,Allansford,3277,Warrnambool (C),142.590393,-38.386281
2,2,Government,Avoca Primary School,Primary,118 Barnett Street,Avoca,3467,Pyrenees (S),143.475649,-37.084502
3,3,Government,Avenel Primary School,Primary,40 Anderson Street,Avenel,3664,Strathbogie (S),145.234722,-36.901368
4,4,Government,Warrandyte Primary School,Primary,5-11 Forbes Street,Warrandyte,3113,Manningham (C),145.213980,-37.742675
...,...,...,...,...,...,...,...,...,...,...
2286,2286,Independent,Youth2Industry College,Secondary,"Level 2, 220 Albert Road",SOUTH MELBOURNE,3205,Port Phillip (C),144.966417,-37.837469
2287,2287,Catholic,St Angela of the Cross Primary School,Primary,181 Mills Road,WARRAGUL,3820,Baw Baw (S),145.961611,-38.141528
2288,2288,Independent,My College,Primary,92 – 96 Railway Crescent,DALLAS,3047,Hume (C),144.923692,-37.675751
2289,2289,Independent,Woodline Primary,Primary,405 Barrabool Road,CERES,3221,Greater Geelong (C),144.285354,-38.171776


In [9]:
poa_sch = []
for i in school["Address_Postcode"]:
    poa_sch.append(i)

poa_school = []
for j in poa_sch:
    if j not in poa_school:
        poa_school.append(j)

len(poa_school)

580

In [14]:
num_school = []
for poa in poa_school: 
    num = 0
    for index,row in school.iterrows():
        if row['Address_Postcode'] == poa:
            num += 1
    
    num_school.append((poa,num))
    
num_school

[(3971, 4),
 (3277, 1),
 (3467, 1),
 (3664, 1),
 (3113, 3),
 (3216, 17),
 (3340, 8),
 (3350, 29),
 (3690, 12),
 (3806, 13),
 (3373, 3),
 (3192, 6),
 (3265, 3),
 (3015, 6),
 (3314, 1),
 (3250, 8),
 (3450, 5),
 (3451, 4),
 (3363, 3),
 (3556, 5),
 (3761, 1),
 (3280, 13),
 (3936, 3),
 (3108, 4),
 (3564, 8),
 (3095, 11),
 (3448, 1),
 (3803, 2),
 (3031, 7),
 (3011, 6),
 (3437, 3),
 (3191, 4),
 (3744, 1),
 (3122, 8),
 (3084, 10),
 (3300, 9),
 (3523, 2),
 (3551, 11),
 (3228, 6),
 (3683, 2),
 (3444, 7),
 (3434, 1),
 (3329, 1),
 (3515, 1),
 (3441, 1),
 (3337, 12),
 (3065, 4),
 (3462, 1),
 (3364, 1),
 (3151, 5),
 (3438, 3),
 (3251, 1),
 (3040, 10),
 (3058, 11),
 (3289, 2),
 (3341, 1),
 (3754, 10),
 (3305, 10),
 (3380, 5),
 (3685, 3),
 (3431, 1),
 (3330, 1),
 (3219, 9),
 (3850, 10),
 (3361, 1),
 (3141, 5),
 (3862, 2),
 (3447, 1),
 (3282, 2),
 (3074, 8),
 (3677, 10),
 (3687, 1),
 (3442, 5),
 (3281, 2),
 (3030, 25),
 (3747, 5),
 (3756, 5),
 (3276, 1),
 (3461, 5),
 (3717, 4),
 (3435, 2),
 (3242, 1),


In [13]:
len(num_school)

580